In [1]:
import fingertips_py as ftp
import requests

In [24]:
# metadata_areas = ftp.metadata.get_all_areas(is_test=False)
# metadata_value_notes = ftp.metadata.get_all_value_notes(is_test=False)
# metadata_profiles = ftp.metadata.get_all_profiles(is_test=False)
# metadata_domains = ftp.metadata.get_domains_in_profile(18)
metadata_all_indicators = ftp.metadata.get_metadata_for_all_indicators_from_csv(is_test=False)
metadata_all_indicator_ids = metadata_all_indicators['Indicator ID'].tolist()

In [27]:
data = ftp.retrieve_data.get_all_data_for_indicators(indicators=[108,
 113,
 114,
 200,
 212],
                                                               area_type_id=7,
                                                               parent_area_type_id=15,
                                                               filter_by_area_codes=None,
                                                               is_test=False)

c:\Users\medssam\Documents\GitHub\ExternalData\.venv\Lib\site-packages\fingertips_py\retrieve_data.py:115: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(base_url + populated_url)


In [33]:
data = ftp.retrieve_data.get_data_by_indicator_ids(indicator_ids=metadata_all_indicator_ids,
                                                   area_type_id=[7,8],
                                                   parent_area_type_id=15,
                                                   profile_id=None,
                                                   include_sortable_time_periods=None,
                                                   is_test=False)

InvalidURL: URL can't contain control characters. '/api/all_data/csv/by_indicator_id?indicator_ids=108,113,114,200,212,219,224,241,247,253,258,262,273,276,280,285,294,295,336,337,342,349,351,352,353,354,355,356,358,359,383,384,639,640,641,642,643,650,811,848,849,1179,1180,1187,1194,1201,1203,1204,1205,1206,1207,1208,1210,1211,1212,1679,1730,1942,10101,10301,10401,10601,10602,11001,11201,11202,11401,11601,20101,20401,20601,20602,21001,22001,22301,22302,22303,22304,22401,22402,22403,30101,30301,30302,30303,30304,30306,30307,30308,30309,30310,30311,30313,30314,30315,30601,40401,40402,40501,40502,40601,40701,40702,41001,41101,41201,41202,41203,41204,41401,41402,41403,90244,90245,90246,90280,90282,90283,90284,90285,90286,90287,90290,90291,90292,90312,90313,90314,90315,90316,90317,90319,90320,90321,90323,90356,90357,90358,90360,90361,90362,90366,90371,90372,90400,90402,90406,90412,90413,90414,90419,90420,90421,90423,90425,90428,90443,90452,90453,90510,90527,90529,90530,90535,90550,90552,9057

In [32]:
data.head()

,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,...,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Compared to percentiles,Time period Sortable,New data,Compared to goal,Time period range
0,114,QOF Total List Size,NaN,NaN,E92000001,England,England,Persons,All ages,NaN,...,54836561,NaN,NaN,NaN,Not compared,Not compared,20090000,NaN,NaN,1y
1,114,QOF Total List Size,NaN,NaN,E92000001,England,England,Persons,All ages,"CCG deprivation deciles in England (IMD2019, 2...",...,508339,NaN,Aggregated from all known lower geography values,NaN,Not compared,Not compared,20090000,NaN,NaN,1y
2,114,QOF Total List Size,E92000001,England,E92000001,England,England,Persons,All ages,NaN,...,54836561,NaN,NaN,NaN,Not compared,Not compared,20090000,NaN,NaN,1y
3,114,QOF Total List Size,E92000001,England,E92000001,England,England,Persons,All ages,"CCG deprivation deciles in England (IMD2019, 2...",...,508339,NaN,Aggregated from all known lower geography values,NaN,Not compared,Not compared,20090000,NaN,NaN,1y
4,114,QOF Total List Size,E92000001,England,A81001,The Densham Surgery,GPs,Persons,All ages,NaN,...,4164,NaN,NaN,NaN,Not compared,Not compared,20090000,NaN,NaN,1y
